## **Installing Required Dependecies**

In [ ]:
!pip install foolbox torch torchvision cleverhans torchattacks adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl 

## **Fine-tune Resnet18 on CIFAR10**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
learning_rate = 0.001
batch_size = 64

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'resnet18_cifar10.pth')

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')


100%|██████████| 170498071/170498071 [00:03<00:00, 43042919.20it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 174MB/s]
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cu

Epoch [1/10], Step [100/782], Loss: 0.6080
Epoch [1/10], Step [200/782], Loss: 0.6426
Epoch [1/10], Step [300/782], Loss: 0.4534
Epoch [1/10], Step [400/782], Loss: 0.4303
Epoch [1/10], Step [500/782], Loss: 0.3247
Epoch [1/10], Step [600/782], Loss: 0.5733
Epoch [1/10], Step [700/782], Loss: 0.3175
Epoch [2/10], Step [100/782], Loss: 0.2260
Epoch [2/10], Step [200/782], Loss: 0.6204
Epoch [2/10], Step [300/782], Loss: 0.2094
Epoch [2/10], Step [400/782], Loss: 0.3095
Epoch [2/10], Step [500/782], Loss: 0.5173
Epoch [2/10], Step [600/782], Loss: 0.2774
Epoch [2/10], Step [700/782], Loss: 0.1408
Epoch [3/10], Step [100/782], Loss: 0.2842
Epoch [3/10], Step [200/782], Loss: 0.2344
Epoch [3/10], Step [300/782], Loss: 0.1734
Epoch [3/10], Step [400/782], Loss: 0.2955
Epoch [3/10], Step [500/782], Loss: 0.2753
Epoch [3/10], Step [600/782], Loss: 0.1329
Epoch [3/10], Step [700/782], Loss: 0.2842
Epoch [4/10], Step [100/782], Loss: 0.0737
Epoch [4/10], Step [200/782], Loss: 0.1246
Epoch [4/10

## **Preparing Model & Dataset**

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import foolbox as fb
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
import torchattacks
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 10)
model.load_state_dict(torch.load('resnet18_cifar10.pth'))
model = model.to(device)
model.eval()

Files already downloaded and verified


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## **General Functions to Compute ASR**

In [ ]:
def predict(model, images):
    logits = model(images)
    return logits.argmax(axis=-1)

def compute_asr(dataloader, model, attack_fn, epsilon):
    total = 0
    success = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        clean_predictions = predict(model, images)
        adv_images = attack_fn(model, images, labels, epsilon)
        adv_predictions = predict(model, adv_images)

        success += (clean_predictions != adv_predictions).sum().item()
        total += images.size(0)

    return success / total

## **Attack Functions Using AT Libraries**

In [ ]:
def foolbox_attack_fn(model, images, labels, epsilon):
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    attack = fb.attacks.FGSM()
    adv_images, _, _ = attack(fmodel, images, labels, epsilons=epsilon)
    return adv_images

def cleverhans_attack_fn(model, images, labels, epsilon):
    adv_images = fast_gradient_method(model, images, epsilon, np.inf)
    return adv_images

def torchattacks_attack_fn(model, images, labels, epsilon):
    attack = torchattacks.FGSM(model, eps=epsilon)
    adv_images = attack(images, labels)
    return adv_images

def art_attack_fn(model, images, labels, epsilon):
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=nn.CrossEntropyLoss(),
        optimizer=torch.optim.Adam(model.parameters(), lr=0.01),
        input_shape=(3, 224, 224),
        nb_classes=10,
    )
    attack = FastGradientMethod(estimator=classifier, eps=epsilon)
    adv_images = attack.generate(x=images.cpu().numpy())
    adv_images = torch.tensor(adv_images).to(device)
    return adv_images

epsilon = 0.03

## **ASR using Foolbox**

In [ ]:
asr_foolbox = compute_asr(dataloader, model, foolbox_attack_fn, epsilon)
print(f'(ASR) with Foolbox FGSM and epsilon {epsilon}: {asr_foolbox * 100:.2f}%')



(ASR) with Foolbox FGSM and epsilon 0.03: 61.46%


## **ASR using CleverHans**

In [ ]:
asr_cleverhans = compute_asr(dataloader, model, cleverhans_attack_fn, epsilon)
print(f'(ASR) with CleverHans FGSM and epsilon {epsilon}: {asr_cleverhans * 100:.2f}%')



(ASR) with CleverHans FGSM and epsilon 0.03: 95.59%


## **ASR using Torchattacks**

In [ ]:
asr_torchattacks = compute_asr(dataloader, model, torchattacks_attack_fn, epsilon)
print(f'(ASR) with Torchattacks FGSM and epsilon {epsilon}: {asr_torchattacks * 100:.2f}%')



(ASR) with Torchattacks FGSM and epsilon 0.03: 61.46%


## **ASR using ART**

In [ ]:
asr_art = compute_asr(dataloader, model, art_attack_fn, epsilon)
print(f'(ASR) with ART FGSM and epsilon {epsilon}: {asr_art * 100:.2f}%')


(ASR) with ART FGSM and epsilon 0.03: 95.65%


## **Realizing Attack Success Using A Sample Input**

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import foolbox as fb
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 10)
model.load_state_dict(torch.load('resnet18_cifar10.pth'))
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

image, label = next(iter(dataloader))
image, label = image.to(device), torch.tensor([label]).to(device)


fmodel = fb.PyTorchModel(model, bounds=(0, 1))
clean_logits = fmodel(image)
clean_prediction = clean_logits.argmax(axis=-1)

true_label = label.item()
predicted_label = clean_prediction.item()
class_names = dataset.classes
print(f'True label: {class_names[true_label]} ({true_label})')
print(f'Clean prediction: {class_names[predicted_label]} ({predicted_label})')

attack = fb.attacks.LinfFastGradientAttack()
epsilon = 0.03
adv_image, _, _ = attack(fmodel, image, label, epsilons=epsilon)

adv_logits = fmodel(adv_image)
adv_prediction = adv_logits.argmax(axis=-1)
adv_predicted_label = adv_prediction.item()
print(f'Adversarial prediction: {class_names[adv_predicted_label]} ({adv_predicted_label})')


Files already downloaded and verified
True label: cat (3)
Clean prediction: cat (3)
Adversarial prediction: deer (4)
